In [3]:
import os
import pandas as pd

# modified visualpriors library
from transforms import VisualPriorRepresentation

from classes_analyse_nodes import (
    ImageDataset,
    Pattern_Generator,
    Activation_Pattern,
    correlate_activation_patterns,
    calculate_dataset_metrics,
    taskonomy_activation_layer_shapes,
    )

import torch
import torch.utils.model_zoo # required to load nets
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor

In [4]:
DATASET_NAMES = ('places1', 'places2', 'oasis')
SCALE_NAMES = ('scale2','scale4','scale8','scale16','scale32')

In [5]:
DATA_PATH = './data_256x256'
BEHAVIOR_PATH = './behavior'
RESULTS_PATH = './results_taskonomy'

In [6]:
#VisualPrior.viable_feature_tasks
MODEL_NAMES = ('autoencoding','depth_euclidean','jigsaw','reshading',
               'edge_occlusion','keypoints2d','room_layout', #'colorization' currently not working
               'curvature','edge_texture','keypoints3d','segment_unsup2d',
               'class_object','egomotion','nonfixated_pose','segment_unsup25d',
               'class_scene','fixated_pose','normal','segment_semantic',
               'denoising','inpainting','point_matching','vanishing_point')

In [7]:
IMAGE_TRANSFORMS = ('untransformed')

In [8]:
# get example activation to retrieve activation shape to init Pattern_Genrator

#setup net
model_name = MODEL_NAMES[0]
VisualPriorRepresentation._load_unloaded_nets([model_name])
net = VisualPriorRepresentation.feature_task_to_net[model_name]

#setup activation extractor
_, eval_nodes = get_graph_node_names(net)
return_nodes = { node:node for node in eval_nodes if "conv" in node or 'fc' in node}
activation_extractor = create_feature_extractor(net, return_nodes=return_nodes)

#setup dummy image
dataset = ImageDataset(os.path.join(DATA_PATH, DATASET_NAMES[0], SCALE_NAMES[0]))
dummy_image = next(iter(dataset))
net_activation = activation_extractor(dummy_image[0])

#get activation shape
activation_shapes = taskonomy_activation_layer_shapes(net_activation)

In [9]:
pat = Pattern_Generator(1, activation_shapes, .22)
pattern = pat.get_subset_pattern(0)

In [10]:
net_activation['conv1'].shape

torch.Size([1, 64, 128, 128])

In [11]:
list(pattern.values())[0].shape

torch.Size([1, 64, 128, 128])

In [12]:
act = Activation_Pattern(net_activation)
act[pattern];

In [13]:
correlate_activation_patterns(act[pattern], act[pattern])

{'conv1': 0.9999999999999982,
 'layer1.0.conv1': 1.0,
 'layer1.0.conv2': 0.9999999999999993,
 'layer1.0.conv3': 0.9999999999999978,
 'layer1.1.conv1': 1.0,
 'layer1.1.conv2': 1.0,
 'layer1.1.conv3': 0.9999999999999976,
 'layer1.2.conv1': 0.9999999999999992,
 'layer1.2.conv2': 1.0,
 'layer1.2.conv3': 1.0,
 'layer2.0.conv1': 0.9999999999999998,
 'layer2.0.conv2': 1.0,
 'layer2.0.conv3': 0.9999999999999993,
 'layer2.1.conv1': 0.9999999999999992,
 'layer2.1.conv2': 1.0,
 'layer2.1.conv3': 1.0,
 'layer2.2.conv1': 0.9999999999999989,
 'layer2.2.conv2': 1.0,
 'layer2.2.conv3': 0.9999999999999994,
 'layer2.3.conv1': 0.9999999999999996,
 'layer2.3.conv2': 0.9999999999999996,
 'layer2.3.conv3': 1.0,
 'layer3.0.conv1': 0.9999999999999993,
 'layer3.0.conv2': 0.9999999999999998,
 'layer3.0.conv3': 1.0,
 'layer3.1.conv1': 0.9999999999999991,
 'layer3.1.conv2': 0.9999999999999998,
 'layer3.1.conv3': 1.0,
 'layer3.2.conv1': 0.999999999999998,
 'layer3.2.conv2': 1.0,
 'layer3.2.conv3': 0.99999999999999

In [15]:
import numpy as np
x = pd.DataFrame(np.full((3,2), np.nan), dtype=float)

In [16]:
from collections import OrderedDict

class IntegrationStorage(object):
    """Handle storing results
    
    1 dataframe per network layer:
        - images x iterations
    """
    
    def __init__(self, layer_names: list, num_images: int, num_subsets: int) -> None:
        """
        Each layer has a seperate DataFrame with images X subsets.
        """
        self.data = OrderedDict(
            (layer_name, pd.DataFrame(np.full((num_images, num_subsets), np.nan), dtype=float))
            for layer_name in layer_names)
        
    def insert_subset_data(self, image_id: int, subset_id: int, integration_values: dict):
        """
        Insert integration values of one image in each layer.
        """
        for layer_name in self.data.keys():
            self.data[layer_name].iloc[image_id, subset_id] = integration_values[layer_name]


In [ ]:
# network, fixed-scale, dataset

#geneate 100 000 random node-subsets
#set up datastructures for saving integration measures

# iterate iamges
    #compute image activation, layerwise (store in class Activation_Pattern)

    # iterate node-subsets
        #retrieve random pattern layerwise (class Pattern_Generator)
        #get integration values for random pattern, layerwise (class Activation_Pattern)
        #save integration value in corresponding datastructure

#compute integration and beauty for every node-subset

# full analysis

In [ ]:
for model_name in MODEL_NAMES:
    print(model_name)

    # Import taskonomy model...
    VisualPriorRepresentation._load_unloaded_nets([model_name])
    net = VisualPriorRepresentation.feature_task_to_net[model_name]
    
    # ...and create activation extractor from it
    _, eval_nodes = get_graph_node_names(net)
    return_nodes = { node:node for node in eval_nodes if "conv" in node or 'fc' in node}
    activation_extractor = create_feature_extractor(net, return_nodes=return_nodes)


    for dataset_name in DATASET_NAMES:
        print(dataset_name)

        for scale_name in SCALE_NAMES:
            
            dataset = ImageDataset(
                os.path.join(DATA_PATH, dataset_name, scale_name))
                        
            correlations, selfsimilarity, l2norm = calculate_dataset_metrics(dataset, activation_extractor)
            
            correlations.fillna(correlations.mean())
            selfsimilarity.fillna(selfsimilarity.mean())
            l2norm.fillna(l2norm.mean())
            
            correlations.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'correlations.csv'), index=False, header=False)
            selfsimilarity.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'selfsimilarity.csv'), index=False, header=False)           
            l2norm.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'l2norm.csv'), index=False, header=False)